In [1]:
%pip install dowhy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 989.5/989.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 9.4 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# imports
import networkx as nx 
import numpy as np
import pandas as pd
from dowhy import gcm

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# loading the data 
depression = pd.read_csv("data/Student Depression Dataset.csv")
depression = depression.dropna()
depression = depression.replace({'Yes': 1, 'No': 0})

data_encoded = pd.get_dummies(depression, drop_first=True)
data_encoded = data_encoded[['Academic Pressure', 'Have you ever had suicidal thoughts ?',
 'Financial Stress', 'City_Ahmedabad', 'City_Bhopal', 'City_Faridabad',
 'City_Hyderabad', 'City_Meerut', 'City_Patna', 'Dietary Habits_Moderate',
 'Dietary Habits_Others', 'Dietary Habits_Unhealthy', 'Depression']]

depression_LASSO_features = data_encoded[['Academic Pressure', 'Have you ever had suicidal thoughts ?',
  'Financial Stress', 'City_Ahmedabad', 'City_Bhopal', 'City_Faridabad',
  'City_Hyderabad', 'City_Meerut', 'City_Patna', 'Dietary Habits_Moderate',
  'Dietary Habits_Others', 'Dietary Habits_Unhealthy', 'Depression']]

depression_LASSO_features = depression_LASSO_features.astype(int)

depression_LASSO_features.head()

/var/folders/_f/x2yqbk0n1fq31vqq80nd_2j40000gn/T/ipykernel_55342/2220563643.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  depression = depression.replace({'Yes': 1, 'No': 0})


,Academic Pressure,Have you ever had suicidal thoughts ?,Financial Stress,City_Ahmedabad,City_Bhopal,City_Faridabad,City_Hyderabad,City_Meerut,City_Patna,Dietary Habits_Moderate,Dietary Habits_Others,Dietary Habits_Unhealthy,Depression
0,5,1,1,0,0,0,0,0,0,0,0,0,1
1,2,0,2,0,0,0,0,0,0,1,0,0,0
2,3,0,1,0,0,0,0,0,0,0,0,0,0
3,3,1,5,0,0,0,0,0,0,1,0,0,1
4,4,1,1,0,0,0,0,0,0,1,0,0,0


In [4]:
depression_LASSO_features['Dietary Habits_Unhealthy'].sum()

10316

In [5]:
# X -> treatment (e.g., sleep duration, dietary habits)
# Y -> city of residence?
# Z -> observation (e.g., depression)

# create "scoring" for dietary habits -> 0 for unhealthy, 
# 1 for moderate, 2 for other 

# add the ones = 1 from unhealthy,
# = 1 from moderate, and = 1 for others. before this, 
# encode them as above.
depression_LASSO_features['Dietary_Score'] = (
    depression_LASSO_features['Dietary Habits_Moderate'] * 1 +
    depression_LASSO_features['Dietary Habits_Others'] * 2 +
    depression_LASSO_features['Dietary Habits_Unhealthy'] * 0
)
X = depression_LASSO_features['Dietary_Score']

city_columns = ['City_Ahmedabad', 'City_Bhopal', 'City_Faridabad',
                'City_Hyderabad', 'City_Meerut', 'City_Patna']

depression_LASSO_features['City_Index'] = depression_LASSO_features[city_columns].idxmax(axis=1)
depression_LASSO_features['City_Index'] = depression_LASSO_features['City_Index'].apply(lambda x: city_columns.index(x) + 1)

Y = depression_LASSO_features['City_Index']
Z = depression_LASSO_features['Depression']

In [6]:
training_data = pd.DataFrame(data=dict(X=X, Y=Y, Z=Z))

In [7]:
causal_model = gcm.InvertibleStructuralCausalModel(nx.DiGraph([('X', 'Y'), ('Y', 'Z')])) # X -> Y -> Z
causal_model.set_causal_mechanism('X', gcm.EmpiricalDistribution())
causal_model.set_causal_mechanism('Y', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))
causal_model.set_causal_mechanism('Z', gcm.AdditiveNoiseModel(gcm.ml.create_linear_regressor()))

In [8]:
gcm.fit(causal_model, training_data)

Fitting causal mechanism of node Z: 100%|██████████| 3/3 [00:00<00:00, 93.86it/s]


In [9]:
gcm.counterfactual_samples(
    causal_model,
    {'X': lambda x: 2},
    observed_data=pd.DataFrame(data=dict(X=[1], Y=[2], Z=[3])))

,X,Y,Z
0,2,2.040501,3.000568
